<a href="https://colab.research.google.com/github/Ritil-Ranjan/Youtube/blob/main/Youtube5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import numpy as np
import pandas as pd
import os
import torch
import re

In [2]:
os.environ["WANDB_DISABLED"] = "true"

In [3]:
uploaded = files.upload()

Saving labelled_sentences2.csv to labelled_sentences2.csv


In [4]:
df1 = pd.read_csv('labelled_sentences2.csv')

In [5]:
!pip uninstall -y peft
!pip uninstall -y transformers
!rm -rf /usr/local/lib/python*/dist-packages/transformers
!pip cache purge
!pip install transformers==4.40.1

Found existing installation: peft 0.15.2
Uninstalling peft-0.15.2:
  Successfully uninstalled peft-0.15.2
Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Files removed: 0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 42.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.1 which is incompatible.


In [6]:
# 1. Install necessary packages
!pip install datasets scikit-learn -q

In [7]:
# 2. Load and preprocess your CSV dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Replace with your actual dataset path
df1 = df1.dropna()
df1 = df1.sample(frac=1, random_state=42).reset_index(drop=True)  # shuffle

# Encode labels
label_encoder = LabelEncoder()
df1['label'] = label_encoder.fit_transform(df1['label'])
# label2id = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
# id2label = {v: k for k, v in label2id.items()}

# Assuming you created label_encoder earlier
label2id = {label: int(idx) for idx, label in enumerate(label_encoder.classes_)}
id2label = {v: k for k, v in label2id.items()}

In [8]:
# 3. Convert to Hugging Face Dataset
from datasets import Dataset

dataset = Dataset.from_pandas(df1[['sentence', 'label']])

In [9]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted")
    }

In [10]:
# 4. Tokenize sentences using bert-base-uncased
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize(example):
    return tokenizer(example['sentence'], truncation=True, padding='max_length', max_length=128)

dataset = dataset.map(tokenize)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/5781 [00:00<?, ? examples/s]

In [11]:
# 5. Train-test split
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset['train']
test_dataset = dataset['test']

In [12]:
# Fix: Convert keys to native Python int
# id2label = {int(k): v for k, v in id2label.items()}
# label2id = {v: int(k) for k, v in id2label.items()}

In [13]:
# 6. Load BERT model for classification
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# 7. Set up training arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [15]:
# 8. Train the model
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics  # ✅ Include this!
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.374000,0.378808,0.875540,0.873242
2,0.283000,0.349681,0.881590,0.878308


TrainOutput(global_step=146, training_loss=0.42563029511334144, metrics={'train_runtime': 106.0922, 'train_samples_per_second': 87.169, 'train_steps_per_second': 1.376, 'total_flos': 306270087487488.0, 'train_loss': 0.42563029511334144, 'epoch': 2.0})

In [16]:
#import transformers
#print(transformers.__version__)  # should be 4.40.1

In [17]:
# 9. Evaluate the model
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.3496810793876648, 'eval_accuracy': 0.881590319792567, 'eval_f1': 0.8783075722856586, 'eval_runtime': 4.4285, 'eval_samples_per_second': 261.265, 'eval_steps_per_second': 4.29, 'epoch': 2.0}


In [18]:
# 10. Inference on new sentence

# def classify_sentence(text):
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
#     outputs = model(**inputs)
#     pred_id = outputs.logits.argmax(dim=1).item()
#     return id2label[pred_id]

def classify_sentence(sentence):
    # Tokenize
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)

    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = logits.argmax().item()

    return id2label[predicted_class_id]

# Example
print(classify_sentence("Please help me with this!"))  # exclamatory?

imperative


In [19]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.3496810793876648, 'eval_accuracy': 0.881590319792567, 'eval_f1': 0.8783075722856586, 'eval_runtime': 4.4483, 'eval_samples_per_second': 260.1, 'eval_steps_per_second': 4.271, 'epoch': 2.0}


In [20]:
model.save_pretrained("sentence_classifier_model2")
tokenizer.save_pretrained("sentence_classifier_model2")

('sentence_classifier_model2/tokenizer_config.json',
 'sentence_classifier_model2/special_tokens_map.json',
 'sentence_classifier_model2/vocab.txt',
 'sentence_classifier_model2/added_tokens.json',
 'sentence_classifier_model2/tokenizer.json')

In [22]:
!zip -r sentence_classifier_model2.zip sentence_classifier_model2/
files.download('sentence_classifier_model2.zip')  # downloads to your computer

  adding: sentence_classifier_model2/ (stored 0%)
  adding: sentence_classifier_model2/tokenizer_config.json (deflated 76%)
  adding: sentence_classifier_model2/config.json (deflated 49%)
  adding: sentence_classifier_model2/model.safetensors (deflated 8%)
  adding: sentence_classifier_model2/special_tokens_map.json (deflated 42%)
  adding: sentence_classifier_model2/tokenizer.json (deflated 71%)
  adding: sentence_classifier_model2/vocab.txt (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
df.drop(columns = ['hindi_ratio', 'Unnamed: 0'], inplace=True)

In [86]:
def sentence_type(text):
  new_list = []
  for sent in text.strip().split('\n'):
    new_list.append(classify_sentence(sent))
  return new_list

In [87]:
df['type2'] = df['text'].apply(sentence_type)

In [88]:
df.to_csv('dhruv_rathee11.csv')

In [89]:
files.download('dhruv_rathee11.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
uploaded = files.upload()

Saving dhruv_rathee10.csv to dhruv_rathee10.csv


In [25]:
df = pd.read_csv('dhruv_rathee10.csv')

In [93]:
df['type2'].value_counts()

,count
type2,
[assertive],7498
[interrogative],1649
"[assertive, assertive]",1264
[imperative],898
"[assertive, assertive, assertive]",414
...,...
"[imperative, assertive, assertive, assertive, assertive, assertive, assertive, assertive]",1
"[assertive, assertive, assertive, assertive, assertive, assertive, assertive, assertive, interrogative, assertive, assertive]",1
"[interrogative, assertive, assertive, assertive, interrogative, assertive]",1


In [59]:
def rectify_type(text):
  text = text[0:-1]
  type_list = text.split(',')
  new_list = []
  for elements in type_list:
    elements = elements[2:-1]
    new_list.append(elements)
  return new_list

In [66]:
df['type'] = df['type'].apply(rectify_type)

In [82]:
ex_words = {'ah','alas','gee','god bless','thank god','may god','ha','jesus','ohh','ohoh','omg','ooh','ouch','wow'}

In [72]:
print(df['text'][60:100])

60    well researched video\nkudos to dhruv for expo...
61    every drop of a soldier's sweat and blood tell...
62    i am a pakistani, your videos have opened my e...
63    believe me, if you were a pakistani youtuber, ...
64    you are a true patriot as we all are\nall indi...
65    i love the way bro you speak .. i'm from pakis...
66    14:03 prayers to the fallen, i hope their soul...
67    dhruv rathee trying hard to get back into the ...
68    here you go rathee is back to please his follo...
69                                    you are an indian
70                                you are too late mate
71    21:33 foreign secretary in place of former sec...
72    you didn't prove anything just another blunder...
73           i thinks some mistake but mostly are facts
74    the rafale shot down articles in international...
75    i am from pakistan and bro your every word is ...
76    "from this video of yours, it's clear that you...
77       I don't know but ceasefire is the main 

In [79]:
def remove_consecutive_full_stops(text):
  new_text = []
  for sent in text.strip().split('\n'):
    sent = re.sub(r'\.{2,}', '.', sent)
    sent = sent.replace('.', '\n')
    for sent1 in sent.strip().split('\n'):
      new_text.append(sent1)
  return "\n".join(new_text)

In [80]:
df['text'] = df['text'].apply(remove_consecutive_full_stops)

In [84]:
for text in df['text']:
  for sent in text.strip().split('\n'):
    for phrase in ex_words:
      if phrase in sent:
        print(sent)
        break

Streaming output truncated to the last 5000 lines.
you have exposed yourself enough and your credibility too
no need for a "response" to pakistani people when we have roasted and exposed you for your previous bullshit
dharuv rathe now become a part of gooodi media
pakistan also offered indian grovement help in investigation of pahalgam attack but india denied why
pakistan has destroyed rafale tell india this too
your video is joke itself has hundreds of logics and proof but you will remain illogical forever: pakistan has sent drones wheres india has sent sweet to pakistan, hafiz saeed is banned in usa and check how many indian banned in other countries including your pm gujarat butcher
pakistan is now occupied by asim muneer establishment and they plotted this pehalgaam with modi & nawaz shareef
having to watch ads 4-5 times in a 20-25 minute video!
what to do with so much money?
the pakistani army attacks the pashtun tribes a lot and claims that india did it or some other excuse
and t

In [75]:
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

df[:1000]

,Unnamed: 0,text,likes,Date,Time,confidence,hindi_ratio2,type
0,0,hail india,0,2025-05-31,07:39:17,[0.16971871],0.0,[assertive]
1,1,well done brother,6,2025-05-31,12:07:48,[0.8321717],0.0,[assertive]
2,2,god bless dhruv rathi,1,2025-05-31,13:45:37,[0.3843325],0.0,[assertive]
3,3,watch (fasil warich ),1,2025-05-31,07:38:51,[0.5893226],0.0,[imperative]
4,4,long live unity india,0,2025-05-31,08:59:36,[0.20433551],0.0,[assertive]
5,5,proud to our armed forces,2,2025-05-31,07:37:49,[0.7334306],0.0,[assertive]
6,6,unity in diversity makes india more rich and more beautiful,0,2025-05-31,09:05:45,[0.8453392],0.0,[assertive]
7,7,proud of south indian,1,2025-05-31,16:30:11,[0.6993358],0.0,[assertive]
8,8,very well said dhruv you just nailed it…,3,2025-06-01,17:35:09,[0.94783944],0.0,[assertive]
9,9,"may humanity, love trumps hatred and bigotry",0,2025-05-31,15:12:40,[0.733443],0.0,[assertive]


In [76]:
pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')